In [2]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone
import bcrypt

DB_NAME = "DB_Final_Project"
DB_USER = "postgres"
DB_PASSWORD = "bear123321a"
DB_HOST = "localhost" 

# Connect to the PostgreSQL database
# conn = psycopg2.connect(
#     dbname = DB_NAME, 
#     user = DB_USER, 
#     password = DB_PASSWORD, 
#     host = DB_HOST
# )
# cur = conn.cursor()

In [3]:
def user_create_account(user_id, password, user_name, user_description, profile_pic, 
        profile_background, birthday, email, country, language, fund, 
        filtering, notification, cookies):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    encoded_password = password.encode('utf-8')
    hashed_password = bcrypt.hashpw(encoded_password, bcrypt.gensalt())

    cur.execute(
        """
        INSERT INTO public."user" 
        (user_id, password_hashed, user_name, user_description, profile_pic, 
        profile_background, birthday, email, country, language, fund, 
        filtering, notification, cookies) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            user_id, 
            hashed_password, 
            user_name, 
            user_description, 
            profile_pic, 
            profile_background, 
            birthday, 
            email, 
            country, 
            language, 
            fund, 
            filtering, 
            notification, 
            cookies
        )
    )

    conn.commit()
    return

In [ ]:
def user_login(user_id, password):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        SELECT password_hashed 
        FROM public."user" 
        WHERE user_id = %s
        """,
        (user_id,)
    )
    conn.commit()
    stored_password_hash = cur.fetchone()[0] # should be 60 characters
    encoded_password = password.encode('utf-8')

    success = False
    if bcrypt.checkpw(encoded_password, stored_password_hash):
        success = True

    return success

In [ ]:
def user_change_settings(user_id, user_name, user_description, profile_pic, 
        profile_background, birthday, email, country, language, fund, 
        filtering, notification, cookies):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        UPDATE public."user" 
        SET 
            user_name = %s,
            user_description = %s,
            profile_pic = %s,
            profile_background = %s,
            birthday = %s,
            email = %s,
            country = %s,
            language = %s,
            fund = %s,
            filtering = %s,
            notification = %s,
            cookies = %s
        WHERE 
            user_id = %s
        """,
        (
            user_name,
            user_description,
            profile_pic,
            profile_background,
            birthday,
            email,
            country,
            language,
            fund,
            filtering,
            notification,
            cookies,
            user_id
        )
    )

    conn.commit()
    return

In [ ]:
def user_add_fund(user_id, amount):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        UPDATE public."user" 
        SET fund = fund + %s
        WHERE user_id = %s FOR UPDATE;
        """,
        (amount, user_id,)
    )
    
    conn.commit()
    return

In [ ]:
def user_search_for_games(keywords):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        SELECT password_hashed 
        FROM public."game" 
        WHERE game_name LIKE %s 
        """,
        (f"%{keywords}%",)
    )
    conn.commit()
    related_games = cur.fetchall()
    
    # related_games is a list
    return related_games

In [ ]:
def user_buy_items(user_id, game_id, item_id, timestamp):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    # update fund
    cur.execute(
        """
        SELECT fund 
        FROM public."user" 
        WHERE user_id = %s FOR UPDATE;
        """,
        (user_id,)
    )
    result = cur.fetchone()
    if result is None:
        raise ValueError("User not found.")
    user_fund = result[0]

    cur.execute(
        """
        SELECT current_price 
        FROM public."game_item" 
        WHERE game_id = %s AND item_id = %s FOR SHARE;
        """,
        (game_id, item_id,)
    )
    result = cur.fetchone()
    if result is None:
        raise ValueError("Game item not found.")
    current_price = result[0]

    if user_fund - current_price < 0:
        conn.rollback()
        raise ValueError("Insufficient funds.")


    # add transaction
    cur.execute(
        """
        INSERT INTO public."buy_item" 
        (buy_item_id, user_id, game_id, item_id, price, timestamp, isCancelled) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (
            buy_item_id, user_id, game_id, item_id, current_price, timestamp, False
        )
    )

    conn.commit()
    return

In [ ]:
def user_uninstall_games():

    return

In [ ]:
def user_add_friends(user_id, friend_id):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        INSERT INTO public."user_friends" 
        (user_id, friend_id) 
        VALUES (%s, %s)
        """,
        (user_id, friend_id)
    )
    cur.execute(
        """
        INSERT INTO public."user_friends" 
        (user_id, friend_id) 
        VALUES (%s, %s)
        """,
        (friend_id, user_id)
    )

    conn.commit()
    return

In [ ]:
def user_add_shopping_cart(user_id, game_id, item_id):
    success = False
    try:
        conn = psycopg2.connect(
            dbname = DB_NAME, 
            user = DB_USER, 
            password = DB_PASSWORD, 
            host = DB_HOST
        )
        cur = conn.cursor()

        cur.execute(
            """
            INSERT INTO public."cart" 
            (user_id, game_id, item_id) 
            VALUES (%s, %s, %s)
            ON CONFLICT (user_id, game_id, item_id) DO NOTHING
            """,
            (user_id, game_id, item_id)
        )

        conn.commit()
        success = True
    except Exception as e:
        print(f"An error occurred in user_add_shopping_cart(): {e}")
        conn.commit().rollback()
        
    return success

In [ ]:
def user_delete_shopping_cart(user_id, game_id, item_id):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        DELETE FROM public."cart"
        WHERE user_id = %s AND game_id = %s AND item_id = %s
        """,
        (user_id, game_id, item_id)
    )

    conn.commit() 
    return

In [ ]:
def user_review_games(review_id, user_id, game_id, rating, text, review_timestamp):
    conn = psycopg2.connect(
        dbname = DB_NAME, 
        user = DB_USER, 
        password = DB_PASSWORD, 
        host = DB_HOST
    )
    cur = conn.cursor()
    
    cur.execute(
        """
        INSERT INTO public."game_reviews" 
        (review_id, user_id, game_id, rating, text, review_timestamp) 
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        (review_id, user_id, game_id, rating, text, review_timestamp)
    )

    conn.commit()
    return

In [ ]:
def user_look_up_for_achievements():

    return